In [4]:
# (a)
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

# Defining s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
key_file = 'framingham.csv'

bucket_object = bucket.Object(key_file)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading csv
heart = pd.read_csv(file_content_stream)

# (b)
heart = heart.dropna()
heart.head(1)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0


In [5]:
# Defining the predictors and target variable
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

# Spliting dataset into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [6]:
# Transforming predictor variables to 0-1 scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [8]:
# Adaboost model
md1 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

# Predicting on test dataset
pred1 = md1.predict_proba(X_test)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

# Computing the recall
recall_score(Y_test, pred1)

1.0

In [9]:
# Random Forest
md2 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

# Predicting on test dataset
pred2 = md2.predict_proba(X_test)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

# Computing the recall
recall_score(Y_test, pred2)

0.9642857142857143

#### I would use Adaboost model to make predictions